## Final Project

In [1]:
# all imports
import pandas as pd                     # data frames
import numpy as np                      # ndarrays
import requests as rq                   # http requests
from bs4 import BeautifulSoup as soup   # beautiful soup for parsing
import regex as re                      # regex operations
from plotnine import *                  # descriptive plotting
from datetime import datetime as dt     # manipulating datetime types

## Get Data

In [5]:
# this requests the data on Movie Production Companies sorted by Worldwide Box Office, high->low
num_url = 'https://www.the-numbers.com/movies/production-companies/#production_companies_overview=od3'
num_rq = rq.get(num_url)
num_soup = soup(num_rq.text, 'lxml') # use 'lxml' instead of 'html.parser' -> problems avoided

ls = []                           # for storing entries and will be converted to data frame
body = num_soup.find('tbody')     # find beginning of useful data
trs = body.findAll('tr')          # extract all html <tbody> row entries 
for entry in trs:                 # each row 
    a = entry.find('a')['href']   # find link of each production company -> used to get all movies later
    a = a[26:]                    # remove excessive URL information -> hard code is ugly, but more concise
    x = entry.text.split('\n')    # get all entry data -> company name, movies made, US Boxoffice, Worldwide
    x = x[1:-1]             # trim data of two empty entries
    r = [a]                       # it has to be like this... even the separate rows
    r.extend(x)
    ls.append(r)                  # try to clearn this before final submission

numbers = pd.DataFrame(ls, columns=['Link','Company','No.of Movies','Domestic','Worldwide']) # create data frame

# Clean Data :
# - use regex to convert Number's format to int
# - compute International Box Office by subtracting Worldwide from Domestic 
numbers[numbers.columns[3:]] = numbers[numbers.columns[3:]].replace(r'[$,*]', '', regex=True).astype(int)
numbers['International'] = numbers['Worldwide'] - numbers['Domestic'] # compute US-International Box Office
display(numbers)

,Link,Company,No.of Movies,Domestic,Worldwide,International
0,/Warner-Bros,Warner Bros.,249,18980559739,43889973703,24909413964
1,/Columbia-Pictures,Columbia Pictures,245,18524810041,41490827169,22966017128
2,/Universal-Pictures,Universal Pictures,254,18444401349,43905018026,25460616677
3,/Walt-Disney-Pictures,Walt Disney Pictures,131,15873663739,38420085125,22546421386
4,/Marvel-Studios,Marvel Studios,62,12974487990,33686785120,20712297130
...,...,...,...,...,...,...
13933,/Matemotja-Productions,Matemotja Productions,1,0,0,0
13934,/Littlebig-Pictures,Littlebig Pictures,1,0,0,0
13935,/9Magnan,9Magnan,1,0,0,0
13936,/Fun-Group-The,The Fun Group,1,0,0,0


### We now have all production companies and their respective box office sales
But we want more information. So, for each company, let's retrieve data on the individual movies. This can be accmomplished using the saved URL extensions, requesting data from those sites and treating that data to build a fresh data frame. 

This is the data we want. There is budget information, date information (for normalizing) and we can drop
incomplete rows from consideration. Overall, this is an improvement in accuracy 

In [3]:
# this base URL is a route for each company
base_url = "https://www.the-numbers.com/movies/production-company"
all_entries = []
top1000 = numbers.head(1000)  # only consider the top 1000 companies

# incoming data needs to be properly treated with regex -> returns list (empty or ready for entry)
# - separate Date and Film Title
# - separete budget, opening weekend, domestic, worldwide
# - regect all entries of inappropriate size

def treat_data(entry):
    ret = []
    t = entry.text
    x = re.split(r'(\w+\s\d+,\s\d{4})',t)[1:]
    if x :                                         # IF date is found
        date = dt.strptime(x[0],'%b %d, %Y').date()   # extract date with regex
        info = re.split(r'\n',x[1:][0])               
        title = info[0]                               # extract title
        money = re.split(r'\$',info[1])[1:]           # extract all monetary information
        if len(money) == 4:                           # IF all monetary information is accounted
            money = [re.sub(r',','',m) for m in money]
            ret.append(date)                            # add date
            ret.append(title)                           # add title
            for m in money: ret.append(m)               # add all money
    return ret                                     # return list, empty or accurate

for link,comp in zip(top1000.Link, top1000.Company): # for each entry in data frame
    full_url = base_url + link        # consruct full link using base
    r = rq.get(full_url)        # request full link
    if r.status_code == 200:    # IF status return is good
        SOUP = soup(r.text, 'lxml') # make soup
        body = SOUP.find('tbody')         # find beginning of useful data
        trs = body.findAll('tr')          # extract all <tr> html tags 
        for entry in trs:                 # each each row entry 
            x = treat_data(entry)            # treat data, returning [] if data is insufficient
            if x != []:                      # if data is good
                all_entries.append([comp]+x) # add company specific entry to total entries
    else: bad_status += 1
                
df = pd.DataFrame(all_entries, columns=['Company','Release Date','Film Title',     # film info
                                        'Budget','Opening','Domestic','Worldwide'] # monetary info
                 )
df[df.columns[3:]] = df[df.columns[3:]].astype(int)     # type cast monetary info as int
df['International'] = df['Worldwide'] - df['Domestic']  # compute International box office
display(df)

,Company,Release Date,Film Title,Budget,Opening,Domestic,Worldwide,International
0,Warner Bros.,2020-02-07,Birds of Prey (And the Fant…,82000000,33010017,84158461,201715591,117557130
1,Warner Bros.,2019-12-25,Just Mercy,25000000,107858,36001502,50401502,14400000
2,Warner Bros.,2019-12-13,Richard Jewell,45000000,4705265,22345542,43100570,20755028
3,Warner Bros.,2019-11-08,Doctor Sleep,45000000,14114124,31581712,71856239,40274527
4,Warner Bros.,2019-11-01,Motherless Brooklyn,26000000,3500454,9277736,18132937,8855201
...,...,...,...,...,...,...,...,...
6934,Aviron Pictures,2017-08-04,Kidnap,21000000,10016323,30718107,34836080,4117973
6935,Bryan Singer,2013-03-01,Jack the Giant Slayer,195000000,27202226,65187603,197687603,132500000
6936,Topic,2017-11-17,"Roman J. Israel, Esq.",22000000,61999,11962712,12967012,1004300
6937,Topic,2015-11-06,Spotlight,20000000,295009,45055776,91902438,46846662


## Minor EDA

- opening weekend contribution -> predictor? -> the better the weekend, the better total sales?
- margin = Worldwide / Budget
- compare 'Numbers' main page to new calculations
- standardize per year
- top grossing films per year
- top companies 

In [4]:
# this section will output a few charts and graphs for brief analysis

# compute totals and compare top10 
h_count = 10 
display(df.groupby('Company').sum().sort_values('Worldwide', ascending=False).head(h_count))
display(numbers.head(h_count))

,Budget,Opening,Domestic,Worldwide,International
Company,,,,,
Universal Pictures,11799150000,5042330796,17594635775,42793859759,25199223984
Warner Bros.,12462400000,5028500552,17681054907,41986893094,24305838187
Columbia Pictures,12970500000,4751741782,17639308906,40108383608,22469074702
Walt Disney Pictures,10777400000,3994001877,14817952942,36869575985,22051623043
Marvel Studios,7659000000,4722030799,12974487990,33686785120,20712297130
Paramount Pictures,8951000000,3303125334,12196378524,29187178443,16990799919
20th Century Fox,6208300000,2296404939,9623294162,24621681018,14998386856
Dune Entertainment,4881500000,1758029879,6301169321,16472650266,10171480945
Legendary Pictures,4963500000,2120031511,6266192351,16226727990,9960535639


,Link,Company,No.of Movies,Domestic,Worldwide,International
0,/Warner-Bros,Warner Bros.,249,18980559739,43889973703,24909413964
1,/Columbia-Pictures,Columbia Pictures,245,18524810041,41490827169,22966017128
2,/Universal-Pictures,Universal Pictures,254,18444401349,43905018026,25460616677
3,/Walt-Disney-Pictures,Walt Disney Pictures,131,15873663739,38420085125,22546421386
4,/Marvel-Studios,Marvel Studios,62,12974487990,33686785120,20712297130
5,/Paramount-Pictures,Paramount Pictures,168,12917237722,30206718829,17289481107
6,/20th-Century-Fox,20th Century Fox,108,10106839331,25232413752,15125574421
7,/Relativity-Media,Relativity Media,117,7363230475,15349403396,7986172921
8,/DreamWorks-Pictures,DreamWorks Pictures,82,6698382998,13372006851,6673623853
9,/Dune-Entertainment,Dune Entertainment,70,6307177998,16498034609,10190856611


In [37]:
# gain some insight
# remember, this has not yet been standardized

df['Opening Contribution'] = df['Opening'] / df['Worldwide']
df['Domestic Contribution'] = df['Domestic'] / df['Worldwide']
df['International Contribution'] = df['International'] / df['Worldwide']
df['Profit'] = df['Worldwide'] - df['Budget']
df['Profit Margin'] = df['Profit'] / df['Budget']
display(df[1::50])

,Company,Release Date,Film Title,Budget,Opening,Domestic,Worldwide,International,Opening Contribution,Domestic Contribution,International Contribution,Profit,Profit Margin
1,Warner Bros.,2019-12-25,Just Mercy,25000000,107858,36001502,50401502,14400000,0.002140,0.714294,0.285706,25401502,1.016060
51,Warner Bros.,2013-08-30,Getaway,18000000,4503892,10501938,11813989,1312051,0.381234,0.888941,0.111059,-6186011,-0.343667
101,Warner Bros.,2008-10-10,Body of Lies,67500000,12884416,39394666,118556530,79161864,0.108677,0.332286,0.667714,51056530,0.756393
151,Warner Bros.,1995-06-16,Batman Forever,100000000,52784433,184031112,336529144,152498032,0.156850,0.546850,0.453150,236529144,2.365291
201,Columbia Pictures,2017-03-24,Life,58000000,12501936,30234022,100929666,70695644,0.123868,0.299555,0.700445,42929666,0.740167
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6701,Cinergi Pictures Entertainment,1998-02-27,An Alan Smithee Film: Burn …,10000000,20000,45779,45779,0,0.436882,1.000000,0.000000,-9954221,-0.995422
6751,Producers Sales Organization,1983-10-07,Never Say Never Again,36000000,10958157,55500000,160000000,104500000,0.068488,0.346875,0.653125,124000000,3.444444
6801,Union Investment Partners,2015-11-20,Secret in Their Eyes,20000000,6652996,20180155,32608019,12427864,0.204029,0.618871,0.381129,12608019,0.630401
6851,Matthew Vaughn,2013-08-16,Kick-Ass 2,28000000,13332955,28795985,63129909,34333924,0.211199,0.456139,0.543861,35129909,1.254640


## Retrieve more accurate data

After determining how to limit our requests (~14k is too much and irrelevant),
for every company considered, we will request the data from their individual file.
That individual file contains information on every film for each production company.


## Questions for expansion
what are a few graphs I could explore with a group to
- generate interest
- indicate that more search is needed and possible

how accurate is this data? 
- when does $0 Worldwide become common? -> page 106
- near the bottom, we see that most companies have no profit. 
- inspect individual companies
- lots of missing data, but consistant sales reported
- why? not many production companies make their budgets public
- lets use the links from earlier to build a more accurate data frame

A note on film budgets
- the buget information that we really want is spare data, and not easily found
- data on how much the props department was paid, how much the director was paid, the marketing staff, electricians, extras, each main actor, the producers, script rights, a screenwriter or a screen doctor
- this data could be much more insightful (director success rates, genre specific information...)

In [186]:
# GET DATA SOURCE

# omdb    - online movie database on individual films
# numbers - monetary production data on production companies and films

# end with two dataframes
# - omdb .... hmmm
# - numbers main page sorted by highest worldwide

# EXPAND ON CURRENT DATA
# omdb
# - for each production company on the main page
#      + save the company name
#      + http request the associated link
#      + create a data frame for all the movies with their budgets
#      + append this data to the master dataframe where all movies with their budgets and production companies are listed

# - rename to Domestic to US-Domestic
# - create new column for US-International = (WorldWide - US)
# - create column for Profit Margin = 